In [ ]:
# run the cell first for all setup

import re
from re import search
import os, sys
from os import walk
import traceback
import subprocess
import pandas as pd

import time
import math
import numpy as np

import sklearn
from sklearn.model_selection import train_test_split
from autogluon.tabular import FeatureMetadata
from autogluon.tabular import TabularDataset, TabularPredictor


os.environ["CUDA_LAUNCH_BLOCKING"]="1"
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"             # CPU only
os.environ["RAY_OBJECT_STORE_ALLOW_SLOW_STORAGE"] = "1"

def column_del(name_columnD,df_temp_del, column_drop_list):
    for i, columndel in enumerate (column_drop_list):
        try:
            df_temp_del=df_temp_del.drop(columns=[columndel], axis=1)
        except:     # KeyError
            print (' ')
    return (df_temp_del)

def dataset_split_trainvalidtest (dataload, trainfilein):
    trainprotein=re.sub('\.csv','',trainfilein)
    traindata_split_filename=trainprotein+'_train.csv'
    validdata_split_filename=trainprotein+'_valid.csv'
    trainsize=0.8 # 0.85 or 0.7
    traindata, validdata = train_test_split(dataload, train_size=trainsize) #random_state=777,
    os.chdir(workpath)
    traindata.to_csv(traindata_split_filename,index=False,encoding='utf-8')
    validdata.to_csv(validdata_split_filename,index=False,encoding='utf-8')
    print ('Split dataset : ', trainfilein, ' -> ', traindata_split_filename,' & ', validdata_split_filename)
    return (traindata, validdata)

def AUTOGLUON_train(workpath,trainfilein,AIlabel_problem_eval,trainname):
    starttime1=time.time()
    os.chdir(workpath)
    AIlabel,problem_type,eval_metric,AgonistAnagonist_label=AIlabel_problem_eval
    trainname1=trainname+'_'
    trainname2=re.sub('GPCR_to_train','',trainname) # Agonist or Antagonist
    dataload=TabularDataset(trainfilein)
    dataload.replace([np.inf, -np.inf], np.nan, inplace=True) ###
    trainproteinname=re.sub(trainname1,'',trainfilein) 
    trainproteinname=re.sub(trainname,'',trainproteinname) 
    trainproteinname=re.sub('\.csv','',trainproteinname) 
    dataload=column_del(name_columnD=trainfilein,df_temp_del=dataload, column_drop_list=['Protein_name','ChEMBL ID','Species','EfficacyValue_percentage','level_0', 'index'])
    shape_x0,shape_y0=dataload.shape
    AGtrain=''
    if shape_x0 >8:
        AGtrain='True'        
        traindata,validdata=dataset_split_trainvalidtest(dataload,trainfilein)
        all_x,all_y=dataload.shape
        shape_x,shape_y =traindata.shape
        shape_x2,shape_y2 =validdata.shape
        timelimit = 'YES'  # 'No' 

        traintimelimit=int(shape_x/2)     
        if traintimelimit > 3.5*60*60:
            if re.search('trainAgonistAntagonist',trainfilein):
                traintimelimit=7*60*60 #for LONG allconcat traintimelimit=60hr
            else:
                traintimelimit = 3.5*60*60  # - for AG-concat or ATG-concat protein

        timestamp=time.strftime("%m%d",time.localtime(time.time()))
        print ('\n Now training: ',trainproteinname,' , training data size: ', shape_x, shape_y,' , test data size: ',shape_x2, shape_y2 , ' , \n Train time: ', (traintimelimit/60) ,' mins  at', time.strftime("%Y/%m/%d %H:%M:%S",time.localtime(time.time())) ,'\n\n\n')
        if timelimit == 'YES':
            if traintimelimit>60:
                float2time=int(traintimelimit/60)    
                ML_path = 'AutogluonTRAIN'+timestamp+'_'+trainname2+'_'+trainproteinname+'_'+str(float2time)+'min'
            else:
                float2time=int(traintimelimit)
                ML_path = 'AutogluonTRAIN'+timestamp+'_'+trainname2+'_'+trainproteinname+'_'+str(float2time)+'sec'
        else:
            timestamp=time.strftime("%Y%m%d_%H%M",time.localtime(time.time()))
            ML_path = 'AutogluonTRAIN'+timestamp+'_'+trainname2+'_'+trainproteinname+'_nolimit'
        
        AGtrain=''
        AGtrain_time=0
        while AGtrain=='' : 
            try:
                predictor = TabularPredictor(label=AIlabel, problem_type=problem_type, eval_metric=eval_metric,path=ML_path, verbosity=3).fit(train_data=traindata, auto_stack=True, time_limit=traintimelimit, excluded_model_types=['KNN'])  
                print('\n', time.strftime("%Y/%m/%d %H:%M:%S",time.localtime(time.time())), '\n Trained ', trainproteinname ,(time.time()-starttime1)/60, ' mins ', '\n\n' )
                AGtrain='True'
            except Exception as e:
                print('\n *** Error: ',e,' \n')
                print ('\n ***  REDO Autogluon train : ',AGtrain_time,' for the file: ', trainfilein)
                print('\n', time.strftime("%Y/%m/%d %H:%M:%S",time.localtime(time.time())), '\n')
                AGtrain_time=AGtrain_time+1
                traintimelimit=traintimelimit+1
                AGtrain=''
            if AGtrain_time > 7:
                AGtrain='False'
                leaderboard=pd.DataFrame()
                predictor=pd.DataFrame()
                print ('\n ***  Fail Autogluon train : ',AGtrain_time,' for the file: ', trainfilein,' -- NG -- ')
            
        if AGtrain=='True':
            try:
                leaderboard=predictor.leaderboard(validdata, extra_metrics=['mse','mae','pearsonr','rmse'], silent=True)
                leaderboard=leaderboard.sort_values(by=['score_val'],ascending=False)
                leaderboard=leaderboard.reset_index(drop=True)
                print ('\n\n',leaderboard,'\n Best model: ', predictor.get_model_best())
                leaderboardfile=ML_path+'/leaderboard_'+trainproteinname+'_size'+str(shape_x)+'.csv'
                os.chdir(workpath)
                leaderboard.to_csv(leaderboardfile,index=False,encoding='utf-8')
                print('\n', time.strftime("%Y/%m/%d %H:%M:%S",time.localtime(time.time())), '\n')
            except Exception as e:
                print('\n *** Error: ',e,' \n')
                print (' * CANNOT do evaluations -> skip', trainfilein,'\n')
                AGtrain='False'
                leaderboard=pd.DataFrame()
                predictor=pd.DataFrame()
    else:
        AGtrain='False'
        ML_path=''
        leaderboard=pd.DataFrame()
        predictor=pd.DataFrame()
        traintimelimit=0
        shape_x=int(shape_x0)
        shape_x2=0
        traintimelimit=0
        trainprotein=re.sub(r'.csv','',trainfilein)
        rename_trainfilein=trainprotein+'_smalldataset.csv'
        cmd='cp '+trainfilein+' '+rename_trainfilein
        print ('\n\n small dataset -> copy file: ', trainfilein, rename_trainfilein, '\n  ', cmd)
        os.chdir(workpath)
        subprocess.run(cmd, shell=True) #, capture_output=True)
        print('\n', time.strftime("%Y/%m/%d %H:%M:%S",time.localtime(time.time())), '\n')
    return(AGtrain,leaderboard,predictor,ML_path,shape_x0,shape_x,shape_x2,traintimelimit)


def HUMANTRAINALLconcat(workpath,substr,trainname):
    os.chdir(workpath)
    k=0
    l=0
    alltrainconcat_file=trainname+'_humanalltrainconcat.csv'
    filenames = next(walk(workpath), (None, None, []))[2]
    concat_file_list=[]
    for i, fff in enumerate (filenames):
        if re.search('all', fff):
            continue
        if re.search('_HUMAN_train.csv', fff):
            concat_file_list.append(fff)
    sorted(concat_file_list)
    print ('concat_file_list: ',concat_file_list, len(concat_file_list))
    df_train_all=pd.DataFrame()
    for k, to_concat_file in enumerate (concat_file_list):
        os.chdir(workpath)
        to_concat_file_ori=to_concat_file 
        df_read=pd.read_csv(to_concat_file_ori, low_memory=False) 
        print ('-------------- concat file: (',l,') ',to_concat_file_ori,len(df_read),' to ', alltrainconcat_file, len(df_train_all))
        if l==0:
            df_train_all=df_read
            l=l+1
            df_read_0=df_read
            column_labels=df_read_0.columns
        else:
            try:
                df_train_all=pd.concat([df_train_all,df_read], axis=0, ignore_index=True)
                l=l+1
            except:
                print ('skip file: ', to_concat_file_ori)

    os.chdir(workpath)
    print ('Total concat proteins: ', l, '\n concat_file_list: ',concat_file_list, len(concat_file_list))
    df_train_all.to_csv(alltrainconcat_file,index=False,encoding='utf-8')
    print ('Columns: ', df_train_all.columns)
    return (alltrainconcat_file)

def toTRAINfiles(workpath,trainnum,filenames,df_protein_train,substr,train_result_summary_file,AIlabel_problem_eval):
    os.chdir(workpath)
    ec50_label,problem_type,eval_metric,AgonistAnagonist_label= AIlabel_problem_eval
    print ('To train files: ', filenames, len(filenames))
    for i, trainfilein in enumerate (filenames):
        os.chdir(workpath)
        print (' ------------------------------ ',trainnum, trainfilein ,' ------------------------------ ')
        if re.search(substr[0], trainfilein):
            if re.search(substr[3], trainfilein):
                trainname=substr[3]
            elif re.search(substr[4], trainfilein):
                trainname=substr[4]
            elif re.search(substr[5], trainfilein):
                trainname=substr[5]
            trainname1=trainname+'_'
            trainproteinname=re.sub(trainname1,'',trainfilein)
            trainproteinname=re.sub('\.csv','',trainproteinname)
        df_protein_train.at[trainnum,'trainnum']="{:.0f}".format(trainnum)
        df_protein_train.at[trainnum,'Name']=trainproteinname
        trainname2=re.sub('GPCR_to_train','',trainname)
        df_protein_train.at[trainnum,'trainname']=trainname2
        df_protein_train.at[trainnum,'Time']=str(time.strftime("%Y%m%d_%H%M",time.localtime(time.time())))

        df_protein_train.to_csv(train_result_summary_file,index=False,encoding='utf-8')
        print ('\n Run AutoGluon: (',trainnum,')', trainfilein,' - ' ,trainproteinname)
        print('\n',time.strftime("%Y/%m/%d %H:%M:%S",time.localtime(time.time())), '\n')

        AGtrain,leaderboard,predictor,ML_path,shape_x0,shape_x,shape_x2,traintimelimit=AUTOGLUON_train(workpath,trainfilein,AIlabel_problem_eval,trainname)

        if AGtrain=='True':
            
            try:
                print (trainfilein, ' size ',shape_x,'\n BEST model: ',predictor.get_model_best(),', score: ', "{:.2f}".format(leaderboard.at[0,'score_val'] ,' \n'))
                df_protein_train.at[trainnum,'ALL_size']="{:.0f}".format(shape_x0)
                df_protein_train.at[trainnum,'TRAIN_size']="{:.0f}".format(shape_x)
                df_protein_train.at[trainnum,'TEST_size']="{:.0f}".format(shape_x2)
                df_protein_train.at[trainnum,'BEST_model']=predictor.get_model_best()
                df_protein_train.at[trainnum,'score_val']="{:.5f}".format(abs(leaderboard.at[0,'score_val']))                
                df_protein_train.at[trainnum,'mean_squared_error']="{:.5f}".format(abs(leaderboard.at[0,'mean_squared_error']))
                df_protein_train.at[trainnum,'pearsonr']="{:.5f}".format(leaderboard.at[0,'pearsonr'])
                df_protein_train.at[trainnum,'mean_absolute_error']="{:.5f}".format(abs(leaderboard.at[0,'mean_absolute_error']))
                df_protein_train.at[trainnum,'root_mean_squared_error']="{:.5f}".format(abs(leaderboard.at[0,'root_mean_squared_error']))
                df_protein_train.at[trainnum,'score_test']="{:.5f}".format(leaderboard.at[0,'score_test'])
                df_protein_train.at[trainnum,'score_val']="{:.5f}".format(leaderboard.at[0,'score_val'])
                df_protein_train.at[trainnum,'traintimelimit_mins']="{:.5f}".format(traintimelimit/60)
            except:
                df_protein_train.at[trainnum,'ALL_size']="{:.0f}".format(shape_x0)
                df_protein_train.at[trainnum,'TRAIN_size']="{:.0f}".format(shape_x)
                df_protein_train.at[trainnum,'BEST_model']='skip'
                
        else:
            print ('\n skip \n\n', trainnum, trainfilein)
            df_protein_train.at[trainnum,'ALL_size']="{:.0f}".format(shape_x0)
            df_protein_train.at[trainnum,'TRAIN_size']="{:.0f}".format(shape_x)
            df_protein_train.at[trainnum,'BEST_model']='skip'
        df_protein_train.at[trainnum,'File_Name']=str(trainfilein)
        trainnum=trainnum+1
        df_protein_train.to_csv(train_result_summary_file,index=False,encoding='utf-8')
        print('\n', time.strftime("%Y/%m/%d %H:%M:%S",time.localtime(time.time())), ' done training: ', trainnum,trainfilein)

    os.chdir(workpath)
    df_protein_train.to_csv(train_result_summary_file,index=False,encoding='utf-8')
    print ('\n ********************************* \n',df_protein_train,'\n ********************************* \n')
    print('\n', time.strftime("%Y/%m/%d %H:%M:%S",time.localtime(time.time())),  ' done training proteins \n', filenames, trainnum, '\n\n\n\n\n\n\n')
    return (df_protein_train,trainnum)



ec50_label='ActivityValue_log_EC50'
AgonistAnagonist_label='AgonistP1AnagonistN1_label'
problem_type ='regression'  
eval_metric='mean_squared_error'

AIlabel_problem_eval=[ec50_label,problem_type,eval_metric,AgonistAnagonist_label]
substr=['GPCR_to_train','_train.csv','_valid.csv','GPCR_to_trainAgonistAntagonist','GPCR_to_trainAgonist','GPCR_to_trainAntagonist']

homepath=str(os.getcwd())






### CODE: Main
##### for separate protein Agonist using Autogluon training


In [ ]:
# separate protein Agonist training

timestamp=time.strftime("%Y%m%d%H%M",time.localtime(time.time()))
starttime=time.time()

train_result_summary_file='protein_train_leaderboard_evaluate_sklearn_'+str(timestamp)+'.csv'

workpath=homepath+'/Human_AgonistAntagonist/GPCR_separatePROTEIN/Agonist'

trainname=substr[4]
os.chdir(workpath)

df_protein_train=pd.DataFrame()
trainnum=0 
################################### to all train separate ##########################################
filenames=[]
pp2skip=[]
trainname1=trainname+'_'
folderfiles=(sorted(next(walk(workpath))[2], reverse = True))
for i, file in enumerate (folderfiles):
    ppname=re.sub(trainname1,'',file)
    if re.search('_train.csv', file):
        ppname=re.sub(substr[1],'',ppname)
        pp2skip.append(ppname)
        continue
    if re.search('_valid.csv', file):
        ppname=re.sub(substr[2],'',ppname)
        pp2skip.append(ppname)
        continue
    if re.search('_smalldataset.csv', file):
        ppname=re.sub('_smalldataset.csv','',ppname)
        pp2skip.append(ppname)
        continue
    if re.search(trainname, file):
        filenames.append(file)
        
filenames=sorted(filenames)
print (filenames,len(filenames))
df_protein_train,trainnum = toTRAINfiles (workpath,trainnum,filenames,df_protein_train,substr,train_result_summary_file,AIlabel_problem_eval)

print(time.strftime("%Y/%m/%d %H:%M:%S",time.localtime(time.time())), '\n Total = ', (time.time()-starttime)/3600, ' hrs \n' )


os.chdir(workpath)
df_protein_train.to_csv(train_result_summary_file,index=False,encoding='utf-8')



print('\n', time.strftime("%Y/%m/%d %H:%M:%S",time.localtime(time.time())), '  \n done all training separate  proteins \n')


###################################  alltrain concat ##########################################
timestamp=time.strftime("%m%d",time.localtime(time.time()))
starttime=time.time()

alltrainconcat_file = HUMANTRAINALLconcat(workpath,substr,trainname)

filenames = [alltrainconcat_file]
df_protein_train,trainnum = toTRAINfiles (workpath,trainnum,filenames,df_protein_train,substr,train_result_summary_file,AIlabel_problem_eval)


os.chdir(workpath)
df_protein_train.to_csv(train_result_summary_file,index=False,encoding='utf-8')


##########################################

if (time.time()-starttime)/3600 > 1 :
    print(time.strftime("%Y/%m/%d %H:%M:%S",time.localtime(time.time())), '\n Total = ', (time.time()-starttime)/3600, ' hrs \n' )
else:
    print(time.strftime("%Y/%m/%d %H:%M:%S",time.localtime(time.time())), '\n Total = ', (time.time()-starttime)/60, ' mins \n' )








### CODE: Main
##### for separate protein Antagonist using Autogluon training


In [ ]:
# separate protein Antagonist training

timestamp=time.strftime("%Y%m%d%H%M",time.localtime(time.time()))
starttime=time.time()

train_result_summary_file='protein_train_leaderboard_evaluate_sklearn_'+str(timestamp)+'.csv'


workpath=homepath+'/Human_AgonistAntagonist/GPCR_separatePROTEIN/Antagonist'

trainname=substr[5]
os.chdir(workpath)

df_protein_train=pd.DataFrame()
trainnum=0 
################################### to all train separate ##########################################
filenames=[]
pp2skip=[]
trainname1=trainname+'_'
folderfiles=(sorted(next(walk(workpath))[2], reverse = True))
for i, file in enumerate (folderfiles):
    ppname=re.sub(trainname1,'',file)
    if re.search('_train.csv', file):
        ppname=re.sub(substr[1],'',ppname)
        pp2skip.append(ppname)
        continue
    if re.search('_valid.csv', file):
        ppname=re.sub(substr[2],'',ppname)
        pp2skip.append(ppname)
        continue
    if re.search('_smalldataset.csv', file):
        ppname=re.sub('_smalldataset.csv','',ppname)
        pp2skip.append(ppname)
        continue
    if re.search(trainname, file):
        filenames.append(file)
        
filenames=sorted(filenames)
print (filenames,len(filenames))
df_protein_train,trainnum = toTRAINfiles (workpath,trainnum,filenames,df_protein_train,substr,train_result_summary_file,AIlabel_problem_eval)

print(time.strftime("%Y/%m/%d %H:%M:%S",time.localtime(time.time())), '\n Total = ', (time.time()-starttime)/3600, ' hrs \n' )


os.chdir(workpath)
df_protein_train.to_csv(train_result_summary_file,index=False,encoding='utf-8')



print('\n', time.strftime("%Y/%m/%d %H:%M:%S",time.localtime(time.time())), '  \n done all training separate  proteins \n')


###################################  alltrain concat ##########################################
timestamp=time.strftime("%m%d",time.localtime(time.time()))
starttime=time.time()

alltrainconcat_file = HUMANTRAINALLconcat(workpath,substr,trainname)

filenames = [alltrainconcat_file]
df_protein_train,trainnum = toTRAINfiles (workpath,trainnum,filenames,df_protein_train,substr,train_result_summary_file,AIlabel_problem_eval)


os.chdir(workpath)
df_protein_train.to_csv(train_result_summary_file,index=False,encoding='utf-8')


##########################################

if (time.time()-starttime)/3600 > 1 :
    print(time.strftime("%Y/%m/%d %H:%M:%S",time.localtime(time.time())), '\n Total = ', (time.time()-starttime)/3600, ' hrs \n' )
else:
    print(time.strftime("%Y/%m/%d %H:%M:%S",time.localtime(time.time())), '\n Total = ', (time.time()-starttime)/60, ' mins \n' )







### CODE: Main
##### for separate protein AgonistAntagonist using Autogluon training


In [ ]:
# separate protein AgonistAntagonist training

timestamp=time.strftime("%Y%m%d%H%M",time.localtime(time.time()))
starttime=time.time()

train_result_summary_file='protein_train_leaderboard_evaluate_sklearn_'+str(timestamp)+'.csv'

workpath=homepath+'/Human_AgonistAntagonist/GPCR_separatePROTEIN/AgonistAntagonist'

trainname=substr[3]
os.chdir(workpath)

df_protein_train=pd.DataFrame()
trainnum=0 

################################### to all train separate ##########################################
filenames=[]
pp2skip=[]
trainname1=trainname+'_'
folderfiles=(sorted(next(walk(workpath))[2], reverse = True))
for i, file in enumerate (folderfiles):
    ppname=re.sub(trainname1,'',file)
    if re.search('_train.csv', file):
        ppname=re.sub(substr[1],'',ppname)
        pp2skip.append(ppname)
        continue
    if re.search('_valid.csv', file):
        ppname=re.sub(substr[2],'',ppname)
        pp2skip.append(ppname)
        continue
    if re.search('_smalldataset.csv', file):
        ppname=re.sub('_smalldataset.csv','',ppname)
        pp2skip.append(ppname)
        continue
    if re.search(trainname, file):
        filenames.append(file)
        
filenames=sorted(filenames)
print (filenames,len(filenames))
df_protein_train,trainnum = toTRAINfiles (workpath,trainnum,filenames,df_protein_train,substr,train_result_summary_file,AIlabel_problem_eval)

print(time.strftime("%Y/%m/%d %H:%M:%S",time.localtime(time.time())), '\n Total = ', (time.time()-starttime)/3600, ' hrs \n' )


os.chdir(workpath)
df_protein_train.to_csv(train_result_summary_file,index=False,encoding='utf-8')



print('\n', time.strftime("%Y/%m/%d %H:%M:%S",time.localtime(time.time())), '  \n done all training separate  proteins \n')


###################################  alltrain concat ##########################################
timestamp=time.strftime("%m%d",time.localtime(time.time()))
starttime=time.time()

alltrainconcat_file = HUMANTRAINALLconcat(workpath,substr,trainname)

filenames = [alltrainconcat_file]
df_protein_train,trainnum = toTRAINfiles (workpath,trainnum,filenames,df_protein_train,substr,train_result_summary_file,AIlabel_problem_eval)


os.chdir(workpath)
df_protein_train.to_csv(train_result_summary_file,index=False,encoding='utf-8')


##########################################

if (time.time()-starttime)/3600 > 1 :
    print(time.strftime("%Y/%m/%d %H:%M:%S",time.localtime(time.time())), '\n Total = ', (time.time()-starttime)/3600, ' hrs \n' )
else:
    print(time.strftime("%Y/%m/%d %H:%M:%S",time.localtime(time.time())), '\n Total = ', (time.time()-starttime)/60, ' mins \n' )
